# Logistic Regression 

In [33]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable
import math
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler
import sys

## Loading dataset from MNIST

In [117]:
train_dataset = dsets.MNIST(root='./data/train',train=True,transform=transforms.ToTensor(),download=True)
valid_dataset = dsets.MNIST(root='./data/valid',train=True,transform=transforms.ToTensor(),download=True)
test_dataset = dsets.MNIST(root='./data/test',train=False,transform=transforms.ToTensor(),download=True)

In [118]:
(len(train_dataset))

60000

## Sampling

In [119]:
dset_size = len(train_dataset)
indices = list(range(dset_size))
train_end = math.floor(0.75*dset_size)
train_idx, valid_idx = indices[:train_end],indices[train_end:]
train_sample = SubsetRandomSampler(train_idx)
valid_sample = SubsetRandomSampler(valid_idx)

## Parameters

In [166]:
batch_size = 128
epochs = 10
learning_rate=0.01

In [167]:
train_features = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,sampler=train_sample)
valid_features = torch.utils.data.DataLoader(valid_dataset,batch_size=batch_size,sampler=valid_sample)
test_features = torch.utils.data.DataLoader(test_dataset,batch_size=batch_size,shuffle=True)

In [168]:
print('train n_batches ',len(train_features))
print('valid n_batches ',len(valid_features))
print('test n_batches',len(test_features))

train n_batches  352
valid n_batches  118
test n_batches 79


## Defining model

In [169]:
class LogisticRegression(nn.Module):
    def __init__(self,input_size,num_classes):
        super(LogisticRegression,self).__init__()
        fc1_out_size = 64
        self.fc1 = nn.Linear(input_size,fc1_out_size)
        self.batch1 = nn.BatchNorm1d(fc1_out_size)
        self.fc2 = nn.Linear(fc1_out_size,num_classes)
        self.batch2 = nn.BatchNorm1d(num_classes)
    
    def forward(self,x):
        fc1 = self.fc1(x)
        fc1 = self.batch1(fc1)
        fc1 = F.softmax(fc1,dim=1)
        fc2 = self.fc2(fc1)
        fc2 = self.batch2(fc2)
        return F.softmax(fc2,dim=1)
        

In [170]:
model = LogisticRegression(input_size=28*28,num_classes=10)

## Defining Loss

In [171]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),learning_rate)

## Training model

In [172]:
iterations = 0
for epoch in range(epochs):
    print('Epoch ',epoch+1)
    for i,(images,labels) in enumerate(train_features):
        
        '''
        As long as the image shape is a 4D tensor(64,1,28,28) we need to reshape it to fit into model which
        is a linear fc with 28*28 input dim size. So the feature dim is (batch_size,28*28).
        '''
        features = Variable(images.view(-1,28*28))
        targets  = Variable(labels)

        optimizer.zero_grad()
        outputs = model(features)
        loss = loss_function(outputs,targets)
        
        #backprop
        loss.backward()
        
        #step
        optimizer.step()
        iterations +=1
                
        if (iterations % 500 == 0) or iterations==(epoch+1)*(len(train_features)) or iterations==1:
            evaluated = 0
            total_correct = 0
            for i, (val_images,val_labels) in enumerate(valid_features):
                val_features = Variable(val_images.view(-1,28*28))
                val_targets = Variable(val_labels)
                eval_model = model.eval()
                val_outs  = eval_model(val_features)
                _, predicted = torch.max(val_outs.data,1)
                total_correct += (predicted==val_labels).sum().numpy()
                evaluated += val_labels.size(0)
            
            acc = (total_correct/evaluated)*100
            print("\r Iteration {:4d} \tLoss {:7.3f} \tAccuracy: {:4.2f} \
            Correct {:4d} Evaluated: {:4d}".format(iterations,loss,acc,total_correct,evaluated),end="")
            sys.stdout.flush()

    print('')

print('Total iterations {:4d}'.format(iterations))          

Epoch  1
 Iteration  352 	Loss   1.749 	Accuracy: 67.04             Correct 10056 Evaluated: 15000
Epoch  2
 Iteration  704 	Loss   1.734 	Accuracy: 68.66             Correct 10299 Evaluated: 15000
Epoch  3
 Iteration 1056 	Loss   1.548 	Accuracy: 90.15             Correct 13522 Evaluated: 15000
Epoch  4
 Iteration 1408 	Loss   1.541 	Accuracy: 90.57             Correct 13585 Evaluated: 15000
Epoch  5
 Iteration 1760 	Loss   1.550 	Accuracy: 90.75             Correct 13613 Evaluated: 15000
Epoch  6
 Iteration 2112 	Loss   1.560 	Accuracy: 90.74             Correct 13611 Evaluated: 15000
Epoch  7
 Iteration 2464 	Loss   1.538 	Accuracy: 90.62             Correct 13593 Evaluated: 15000
Epoch  8
 Iteration 2816 	Loss   1.558 	Accuracy: 90.81             Correct 13621 Evaluated: 15000
Epoch  9
 Iteration 3168 	Loss   1.530 	Accuracy: 91.17             Correct 13675 Evaluated: 15000
Epoch  10
 Iteration 3520 	Loss   1.514 	Accuracy: 90.97             Correct 13645 Evaluated: 15000
Total ite

## Running model against test dataset

In [173]:
evaluated = 0
total_correct = 0
model = model.eval()
for i, (val_images,val_labels) in enumerate(test_features):
    val_features = Variable(val_images.view(-1,28*28))
    val_targets = Variable(val_labels)
    val_outs  = model(val_features)
    _, predicted = torch.max(val_outs.data,1)
    total_correct += (predicted==val_labels).sum().numpy()
    evaluated += val_labels.size(0)
test_accuracy = 100.0
test_accuracy *=  total_correct/evaluated
print('Accuracy is {:7.2f}%'.format(test_accuracy))

Accuracy is   91.28%
